In [1]:
import os
import sys
os.chdir('../')
SCRIPDIR = os.path.dirname(os.path.abspath("test.ipynb"))
sys.path.append(os.path.dirname(SCRIPDIR))
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def pick_a_sample(check_df: pd.DataFrame, result_log: pd.DataFrame = None):
    if check_df[check_df['ERROR_REASON'].isna()].shape[0] == 0:
        print("No error found!")
        return None
    sample = check_df[check_df['ERROR_REASON'].isna()].sample()
    if result_log is not None:
        print("-----------I have the schema generated as follows:---------")
        logs = result_log.iloc[sample.LOGS_INDEX.values[0]]
        # temp_df = df[df['TESTFILE_PATH'] == sample['TESTFILE_PATH'].values[0]]
        # for sql in temp_df[temp_df['CASE_TYPE'] == 'Statement']['SQL']:
        #     print(sql)
        print(logs.values[0])
    print("-----------The SQL commands is:---------")
    print(sample.SQL.values[0])
    print("-----------The expected result is:---------")
    print(sample.EXPECTED_RESULT.values[0])
    print("-----------The actual result is:---------")
    print(sample.ACTUAL_RESULT.values[0])
    print("-----------The error message is:---------")
    print(sample.ERROR_MSG.values[0])

    print("-----------The test file is:---------")
    print(sample.TESTFILE_PATH.values[0])
    print("___________The index is:___________")
    print(sample.index.values[0])
    return sample.index.values[0]

# DuckDB

In [ ]:
dbms_name = 'duckdb'

In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_sqlite_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_sqlite_logs.csv")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, result_log)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'INT_DIV' : lambda x: re.search('/', str(x['SQL'])) is not None and x['ERROR_MSG'] == 'Result MisMatch', 
    'VARCHAR_SYNTAX': lambda x: str.startswith(str(x['ERROR_MSG']), "Binder Error: No function matches the given name and argument types '+(VARCHAR)'"),
    'COL_IN_AGG': lambda x: re.match(r"Binder Error: column .* must appear in the GROUP BY clause", x['ERROR_MSG']) is not None,
    'TRIGGER': lambda x: re.match(r"CREATE TRIGGER|DROP TRIGGER", x['SQL'], re.IGNORECASE) is not None,
    # 'UPDATE_MUL': lambda x: re.match(r"Binder Error: Multiple assignments to same column", x['ERROR_MSG']) is not None,
    'INTEGER': lambda x: re.match(
        r"Out of Range Error: Overflow in multiplication of INT32 ", x['ERROR_MSG']) is not None,
    'REPLACE': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_replace.test',
    'UPDATE_MUL': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_update.test',
    'REINDEX': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_reindex.test',
    'EMPTY_SET': lambda x: re.match(r'Parser Error: .* "\)"', x['ERROR_MSG']) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/sqlite_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['REASON'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])

In [ ]:
sample_index = pick_a_sample(check_df, result_log)

In [ ]:
# Manually reason the remaining errors
manual_reason = "REPLACE"

new_df.loc[sample_index, 'ERROR_REASON'] = manual_reason
check_df.loc[sample_index, 'ERROR_REASON'] = manual_reason

In [ ]:
new_df.to_csv(f"output/{dbms_name}_sqlite_results_after.csv", index=False)

In [ ]:
# count the number of errors for each reason in new_df
new_df = pd.read_csv(f"output/{dbms_name}_sqlite_results_after.csv")
new_df.loc[new_df['IS_ERROR'] == False, 'ERROR_REASON'] = 'SUCCESS'
reason_df = new_df[['SQL', 'EXPECTED_RESULT', 'ACTUAL_RESULT','ERROR_REASON', 'IS_ERROR']]
reason_summary = reason_df['ERROR_REASON'].value_counts()
reason_summary.plot(kind='bar')
plt.xlabel("Error Reasons")
plt.ylabel("Number of Errors")
plt.title("SQLite - DuckDB Test Suite Error Reasons Distribution")
# plt.axis(rotation=45)

# PostgreSQL

In [ ]:
dbms_name = 'postgresql'

In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_sqlite_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_sqlite_logs.csv")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, result_log)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'VARCHAR_SYNTAX': lambda x: re.match(r"operator does not exist: \+ text", x['ERROR_MSG']) is not None,
    'VIEW_DEPENDENCY': lambda x: re.match(r"cannot drop view .* because other objects depend on it", x['ERROR_MSG']) is not None,
    'DIV_ZERO': lambda x: re.match(r"division by zero", x['ERROR_MSG']) is not None,
    'TRIGGER': lambda x: re.match(r"CREATE TRIGGER|DROP TRIGGER", x['SQL']) is not None and re.match(r"syntax error", x['ERROR_MSG']) is not None,
    'NULLIF': lambda x: re.search(r"NULLIF", x['SQL']) is not None,
    'CAST': lambda x: re.search(r"CAST", x['SQL']) is not None,
    'COALESCE': lambda x: re.search(r"COALESCE", x['SQL']) is not None,
    'INTEGER': lambda x: re.search(r"integer out of range", x['ERROR_MSG']) is not None,
    'EMPTY_SET': lambda x: re.match(r'syntax error at or near "\)"', x['ERROR_MSG']) is not None,
    'REPLACE': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_replace.test',
    'UPDATE_MUL': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_update.test',
    'REINDEX': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_reindex.test',
    'TYPE_CAST': lambda x: re.search(r"SELECT.*'.*'.*IN\s*\(.+\)", x['SQL']) is not None,
    'CASE_END': lambda x: re.search(r"CASE.*END", x['SQL']) is not None,
    'OPERATE_VIEW': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_createview.test' and x['ACTUAL_RESULT'] == 'True',
}

new_df = copy(df)
reasons = pd.read_csv("data/sqlite_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['REASON'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])

In [ ]:
sample_index = pick_a_sample(check_df, result_log)

In [ ]:
# Manually reason the remaining errors
manual_reason = "REPLACE"

new_df.loc[sample_index, 'ERROR_REASON'] = manual_reason
check_df.loc[sample_index, 'ERROR_REASON'] = manual_reason

In [ ]:
new_df.to_csv(f"output/{dbms_name}_sqlite_results_after.csv", index=False)

In [ ]:
# count the number of errors for each reason in new_df
new_df = pd.read_csv(f"output/{dbms_name}_sqlite_results_after.csv")
new_df.loc[new_df['IS_ERROR'] == False, 'ERROR_REASON'] = 'SUCCESS'
reason_df = new_df[['SQL', 'EXPECTED_RESULT', 'ACTUAL_RESULT','ERROR_REASON', 'IS_ERROR']]
reason_summary = reason_df['ERROR_REASON'].value_counts()
reason_summary.plot(kind='bar')
plt.xlabel("Error Reasons")
plt.ylabel("Number of Errors")
plt.title("SQLite - PostgreSQL Test Suite Error Reasons Distribution")
# plt.axis(rotation=45)

# MySQL

In [ ]:
dbms_name = 'mysql'

In [ ]:
# open a csv file and read the content
df = pd.read_csv(f"output/{dbms_name}_sqlite_results.csv")
result_log = pd.read_csv(f"output/{dbms_name}_sqlite_logs.csv")

# add empty column ERROR_REASON to df
df['ERROR_REASON'] = None
check_df = df.copy()

In [ ]:
pick_a_sample(check_df, result_log)

In [ ]:
from src import utils
from copy import copy
import re

TEMP_FILTER = {
    'COL_IN_AGG': lambda x: re.match(r"1055 \(42000\)", x['ERROR_MSG']) is not None,
    # 'VARCHAR_SYNTAX': lambda x: re.match(r"operator does not exist: \+ text", x['ERROR_MSG']) is not None,
    # 'VIEW_DEPENDENCY': lambda x: re.match(r"cannot drop view .* because other objects depend on it", x['ERROR_MSG']) is not None,
    # 'DIV_ZERO': lambda x: re.match(r"division by zero", x['ERROR_MSG']) is not None,
    'TRIGGER': lambda x: re.match(r"CREATE TRIGGER|DROP TRIGGER", x['SQL']) is not None,
    # 'NULLIF': lambda x: re.search(r"NULLIF", x['SQL']) is not None,
    # 'CAST': lambda x: re.search(r"CAST", x['SQL']) is not None,
    # 'COALESCE': lambda x: re.search(r"COALESCE", x['SQL']) is not None,
    # 'INTEGER': lambda x: re.search(r"integer out of range", x['ERROR_MSG']) is not None,
    # 'EMPTY_SET': lambda x: re.match(r'syntax error at or near "\)"', x['ERROR_MSG']) is not None,
    # 'REPLACE': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_replace.test',
    # 'UPDATE_MUL': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_update.test',
    # 'REINDEX': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_reindex.test',
    # 'TYPE_CAST': lambda x: re.search(r"SELECT.*'.*'.*IN\s*\(.+\)", x['SQL']) is not None,
    # 'CASE_END': lambda x: re.search(r"CASE.*END", x['SQL']) is not None,
    'OPERATE_VIEW': lambda x: x['TESTFILE_PATH'] == 'sqlite_tests/evidence/slt_lang_createview.test' and x['ACTUAL_RESULT'] == True,
    'TIMEOUT': lambda x: re.match(r"Time Exceed", x['ERROR_MSG']) is not None,
    'DROP_INDEX': lambda x: re.match(r"DROP INDEX", x['SQL'], re.IGNORECASE) is not None,
}

new_df = copy(df)
reasons = pd.read_csv("data/sqlite_suite_errors.csv")
tags = reasons[reasons['DBMS'] == dbms_name]['TAG'].values.tolist()
for tag in tags:
    if tag not in TEMP_FILTER:
        print(f"Tag {tag} is not in the test filter, implement it first!")
        continue
    new_df.loc[new_df.apply(TEMP_FILTER[tag], axis=1) & new_df['IS_ERROR'] == True, 'ERROR_REASON'] = tag
    print(tag)
    print(reasons[reasons['TAG']==tag]['REASON'].values[0])
    # print(new_df[new_df['ERROR_REASON']==tag].info())
    print(new_df[new_df['ERROR_REASON']==tag].shape[0])

check_df = copy(new_df[new_df['IS_ERROR'] == True & new_df['ERROR_REASON'].isna()])

In [ ]:
sample_index = pick_a_sample(check_df, result_log)

In [ ]:
# Manually reason the remaining errors
manual_reason = "REPLACE"

new_df.loc[sample_index, 'ERROR_REASON'] = manual_reason
check_df.loc[sample_index, 'ERROR_REASON'] = manual_reason

In [ ]:
new_df.to_csv(f"output/{dbms_name}_sqlite_results_after.csv", index=False)

In [ ]:
# count the number of errors for each reason in new_df
new_df = pd.read_csv(f"output/{dbms_name}_sqlite_results_after.csv")
new_df.loc[new_df['IS_ERROR'] == False, 'ERROR_REASON'] = 'SUCCESS'
reason_df = new_df[['SQL', 'EXPECTED_RESULT', 'ACTUAL_RESULT','ERROR_REASON', 'IS_ERROR']]
reason_summary = reason_df['ERROR_REASON'].value_counts()
reason_summary.plot(kind='bar')
plt.xlabel("Error Reasons")
plt.ylabel("Number of Errors")
plt.title("SQLite - PostgreSQL Test Suite Error Reasons Distribution")
# plt.axis(rotation=45)

In [ ]:
reasons = pd.read_csv("data/sqlite_suite_errors.csv")

duckdb_results = pd.read_csv("output/duckdb_sqlite_results_after.csv")
reason_df = duckdb_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
duckdb_reasons = reason_df['ERROR_REASON'].value_counts()


postgresql_results = pd.read_csv("output/postgresql_sqlite_results_after.csv")
reason_df = postgresql_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
postgresql_reasons = reason_df['ERROR_REASON'].value_counts()

# mysql
mysql_results = pd.read_csv("output/mysql_sqlite_results_after.csv")
reason_df = mysql_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
mysql_reasons = reason_df['ERROR_REASON'].value_counts()

print(duckdb_reasons)
print(postgresql_reasons)
# outer join two dataframes

join_df = pd.concat([duckdb_reasons, postgresql_reasons, mysql_reasons], axis=1, join='outer')
# print(join_df)

join_df.fillna(0, inplace=True)
join_df = join_df.astype(int)

# pure_reasons = reasons[['TAG', 'REASON']].drop_duplicates()
pure_reasons = reasons[['TAG', 'SUPER_TAG', 'REASON']].drop_duplicates()
# join the join_df and the reasons dataframe, drop duplicate rows
join_df = join_df.join(pure_reasons.set_index('TAG'),  how='left')
join_df.sort_values(by=['SUPER_TAG'], inplace=True)
# sum the rows with the same SUPER_TAG
join_df.reset_index(inplace=True, drop=True)
join_df = join_df.groupby('SUPER_TAG').sum()
join_df.to_latex("output/error_reasons_highlevel.tex", )
# join_df.style.to_latex("output/error_reasons.tex", )


In [13]:
reasons = pd.read_csv("data/sqlite_suite_errors.csv")

duckdb_results = pd.read_csv("output/duckdb_sqlite_results_after.csv")
reason_df = duckdb_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
duckdb_reasons = reason_df['ERROR_REASON'].value_counts()


postgresql_results = pd.read_csv("output/postgresql_sqlite_results_after.csv")
reason_df = postgresql_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
postgresql_reasons = reason_df['ERROR_REASON'].value_counts()

# mysql
mysql_results = pd.read_csv("output/mysql_sqlite_results_after.csv")
reason_df = mysql_results[['SQL', 'EXPECTED_RESULT',
                    'ACTUAL_RESULT', 'ERROR_REASON', 'IS_ERROR']]
mysql_reasons = reason_df['ERROR_REASON'].value_counts()

print(duckdb_reasons)
print(postgresql_reasons)
# outer join two dataframes

join_df = pd.concat([duckdb_reasons, postgresql_reasons, mysql_reasons], axis=1, join='outer')
# print(join_df)

join_df.fillna(0, inplace=True)
join_df = join_df.astype(int)

# pure_reasons = reasons[['TAG', 'REASON']].drop_duplicates()
pure_reasons = reasons[['TAG', 'SUPER_TAG']].drop_duplicates()
print(pure_reasons['SUPER_TAG'].value_counts())
# join the join_df and the reasons dataframe, drop duplicate rows
join_df = join_df.join(pure_reasons.set_index('TAG'),  how='left')
join_df.sort_values(by=['SUPER_TAG'], inplace=True)

detail_df = join_df.copy()

detail_df.drop(columns=['ERROR_REASON'], inplace=True)
# sum the rows with the same SUPER_TAG
join_df.reset_index(inplace=True, drop=True)

# unique_reasons = join_df['SUPER_TAG'].drop_duplicates()
join_df = join_df.groupby('SUPER_TAG').sum()
join_df.to_latex("output/error_reasons_highlevel.tex", )
# join_df.style.to_latex("output/error_reasons.tex", )


output_df = detail_df.value_counts()

# unique_reasons = join_df['SUPER_TAG'].drop_duplicates()
# print(unique_reasons)
schema = "\\newcommand{\\SQLiteCompatibilityIssue%s}{%s}\n"

with open("output/Variables.tex", "a") as f:

    f.write(schema % ('SQL',  output_df['Command']))
    # f.write(schema % ('Function', output_df['UFunction']))
    f.write(schema % ('Type', output_df['Type']))
    f.write(schema % ('Operator', output_df['Operator']))
    # f.write(schema % ('Configuration', output_df['Setting']))
    f.write(schema % ('Semantic', output_df['Function'] +
            output_df['IOperator']))
    # f.write(schema % ('Misc', output_df['Misc']))


INT_DIV           104033
VARCHAR_SYNTAX      7075
COL_IN_AGG          1278
INTEGER               43
TRIGGER               23
EMPTY_SET              8
REPLACE                4
UPDATE_MUL             3
REINDEX                1
Name: ERROR_REASON, dtype: int64
VARCHAR_SYNTAX     6065
VIEW_DEPENDENCY    4860
CAST                387
CASE_END            114
COALESCE             82
NULLIF               26
TRIGGER              23
INTEGER              12
REPLACE               8
EMPTY_SET             8
TYPE_CAST             4
UPDATE_MUL            3
OPERATE_VIEW          2
REINDEX               1
Name: ERROR_REASON, dtype: int64
Command      9
Function     4
Operator     3
IOperator    1
Type         1
Issue        1
Name: SUPER_TAG, dtype: int64


/tmp/ipykernel_1418548/1453108505.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  join_df.to_latex("output/error_reasons_highlevel.tex", )
